In [1]:
import sqlalchemy as db
import sqlite3 as sqlite
import os

In [11]:
engine = db.create_engine('sqlite:///' + os.path.abspath(DB_PATH))

In [12]:
conn = engine.connect()

In [16]:
metadata = db.MetaData()

emp = db.Table('emp', metadata,
              db.Column('Id', db.Integer()),
              db.Column('name', db.String(255), nullable=False),
              db.Column('salary', db.Float(), default=100.0),
              db.Column('active', db.Boolean(), default=True)
              )
metadata.create_all(engine)

In [17]:
#Inserting many records at ones
query = db.insert(emp) 
values_list = [{'Id':'2', 'name':'ram', 'salary':80000, 'active':False},
               {'Id':'3', 'name':'ramesh', 'salary':70000, 'active':True}]
ResultProxy = conn.execute(query,values_list)

In [18]:
import pandas as pd

results = conn.execute(db.select([emp])).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(4)

,Id,name,salary,active
0,2,ram,80000.0,False
1,3,ramesh,70000.0,True


In [12]:
from pathlib import Path

DB_PATH = os.path.join('..', 'data', 'movie_metadata.db')
DB_URL = 'sqlite://' + str(Path(DB_PATH).resolve())


import sqlite3 as sqlite

def create_sqlite_db(path: str):
    conn = None
    try:
        conn = sqlite.connect(path)
    except sqlite.Error as error:
        raise error
    finally:
        if conn:
            conn.close()

create_sqlite_db(DB_PATH)

In [13]:
conn = sqlite.connect(DB_PATH) 
c = conn.cursor()

c.execute('''
          CREATE TABLE IF NOT EXISTS products
          ([product_id] INTEGER PRIMARY KEY, [product_name] TEXT)
          ''')
          
c.execute('''
          CREATE TABLE IF NOT EXISTS prices
          ([product_id] INTEGER PRIMARY KEY, [price] INTEGER)
          ''')
                     
conn.commit()

In [14]:
c.execute('''
          INSERT INTO products (product_id, product_name)

                VALUES
                (1,'Computer'),
                (2,'Printer'),
                (3,'Tablet'),
                (4,'Desk'),
                (5,'Chair')
          ''')

c.execute('''
          INSERT INTO prices (product_id, price)

                VALUES
                (1,800),
                (2,200),
                (3,300),
                (4,450),
                (5,150)
          ''')

conn.commit()

In [16]:
import pandas as pd

c = conn.cursor()
                   
c.execute('''
          SELECT
          a.product_name,
          b.price
          FROM products a
          LEFT JOIN prices b ON a.product_id = b.product_id
          ''')

df = pd.DataFrame(c.fetchall(), columns=['product_name','price'])
print(df)

  product_name  price
0     Computer    800
1      Printer    200
2       Tablet    300
3         Desk    450
4        Chair    150


In [20]:
c.execute('SELECT MAX(price) from prices').fetchall()

[[(800,)]]